In [11]:
import numpy as np
from PIL import Image
import re
from css_parser import *
from fpdf import FPDF
from color_utils import *
import os.path

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
#http://www.fpdf.org/en/script/script74.php
class AlphaPDF (FPDF):
    def __init__ (self, orientation = 'P', unit = 'mm', format = 'A4'):
        self.__extgstates = []
        super().__init__ (orientation, unit, format)
        self.bm = 'Normal'
        self.alpha = 1
    # alpha: real value from 0 (transparent) to 1 (opaque)
    # bm:    blend mode, one of the following:
    #          Normal, Multiply, Screen, Overlay, Darken, Lighten, ColorDodge, ColorBurn,
    #          HardLight, SoftLight, Difference, Exclusion, Hue, Saturation, Color, Luminosity
    def set_alpha(self, alpha, bm='Normal'):
        # set alpha for stroking (CA) and non-stroking (ca) operations
        self.bm = bm
        self.alpha = alpha

        data = {'ca':alpha,'CA':alpha,'BM':'/' + bm, 'n' : 0}
        gs = self.add_ext_gstate(data)
        self.set_ext_gstate(gs + 1)

    def add_page(self, orientation=''):
        # do normal add page stuff
        super().add_page(orientation)
        self.set_alpha(self.alpha, self.bm)
        

    def add_ext_gstate(self, data):
        n = len(self.__extgstates)
        self.__extgstates.append(data)
        return n

    def _enddoc(self):
        if len(self.__extgstates) > 0 and self.pdf_version < '1.4':
            self.pdf_version ='1.4'
        super()._enddoc()

    def set_ext_gstate(self, gs):
        self._out('/GS%d gs' % (gs))

    def _putextgstates(self):
        i = 0
        while i < len(self.__extgstates):
            self._newobj()
            self._out('<</Type /ExtGState')
            self.__extgstates[i]["n"] = self.n
            parms = self.__extgstates[i]
            self._out('/ca %.3F' % (parms["ca"]))
            self._out('/CA %.3F' % (parms["CA"]))
            self._out('/BM ' + parms["BM"])
            self._out('>>')
            self._out('endobj')
            i += 1

    def _putresourcedict(self):
        super()._putresourcedict()
        self._out('/ExtGState <<')
        for index, eg in enumerate(self.__extgstates):
            self._out('/GS' + str(index + 1) + ' ' + str(eg["n"])  + ' 0 R')
        self._out('>>')

    def _putresources(self):
        self._putextgstates()
        super()._putresources()
        
class ColoredPDF (AlphaPDF):
    
    def __init__(self, orientation='P',unit='mm',format='A4'):
        super().__init__(orientation, unit, format)
        self.background_color = ""

    #hex color
    def set_background_color(self, color):
        if type(color) in (tuple, list, np.array) and (len(color) == 3 or len(color) == 4):
            color = rgb_to_hex(color)

        self.background_color = color
        
    def set_fill_color_str(self, fill_color):
        self.fill_color = fill_color
        self.color_flag=(self.fill_color!=self.text_color)
        if(self.page>0):
            self._out(self.fill_color)

    def add_page(self, orientation=''):
        # do normal add page stuff
        super().add_page(orientation)

        if self.background_color:
            # before pdf did it's content placement stuff insert background
            rgb = hex_to_rgba(self.background_color)

            last_alpha = self.alpha
            last_bm = self.bm
            last_fill = self.fill_color

            self.set_alpha(1)

            self.set_fill_color(rgb[0], rgb[1], rgb[2])
            self.rect(0,0, 210, 297, "F")

            self.set_alpha(last_alpha, last_bm)
            self.set_fill_color_str(last_fill)

In [116]:
def try_convert_color(color):
    if type(color) in (str):
        color = rgb_to_hex(color)
        
class TextStyle:
    def __init__(self, font_size = 16, font_color = [5, 5, 5], background_color = None):
        self.font_size = font_size
        self.font_color = font_color
        self.background_color = background_color

class DocStyle:
    def __init__(self, background_color = [200, 200, 200]):
        self.background_color = background_color


class StyledPDF(ColoredPDF):
    def __init__(self, orientation='P',unit='mm',format='A4'):
        super().__init__(orientation, unit, format)

        self.set_doc_style(DocStyle())

        self.set_text_style(TextStyle())

    def set_doc_style(self, style: DocStyle):
        self.doc_style = style
        self.set_background_color(style.background_color)
        
    def set_text_style(self, style: TextStyle):
        self.text_style = style
        print(style.font_color)
        self.set_font("Arial", size = style.font_size)
        self.set_text_color(style.font_color[0], style.font_color[1], style.font_color[2])

        if style.background_color is not None:
            self.set_fill_color(style.background_color[0], style.background_color[1], style.background_color[2])
        else:
            color = self.doc_style.background_color
            self.set_fill_color(color[0], color[1], color[2])

    def bullet(self):
        self.cell(1, 7, '.', align =  'R')


In [118]:
input_file = "MDtest.md"
f = open(input_file, "r")

pdf = StyledPDF() 

pdf.add_page()


for x in f:
    cell_h = 10

    pdf.cell(150, cell_h, txt = x, ln = 1, align = 'L', fill= 1)

pdf.output("mygfg.pdf")   

[5, 5, 5]


''